In [3]:
import cv2
import numpy as np
import time
from dbr import *

In [2]:
pip install opencv-contrib-python

  Obtaining dependency information for opencv-contrib-python from https://files.pythonhosted.org/packages/aa/2e/576ac47f21d555b459ca837bb3fb937e50339b8fbfd294945ea2f5290416/opencv_contrib_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/45.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/45.3 MB 660.6 kB/s eta 0:01:09
   ---------------------------------------- 0.1/45.3 MB 812.7 kB/s eta 0:00:56
   ---------------------------------------- 0.1/45.3 MB 751.6 kB/s eta 0:01:01
   ---------------------------------------- 0.2/45.3 MB 1.3 MB/s eta 0:00:37
   ---------------------------------------- 0.3/45.3 MB 1.1 MB/s eta 0:00:41
   ---------------------------------------- 0.4/45.3 MB 1.3 MB/s eta 0:00:35
   ---------------------------------------- 0.4/45.3 MB 1.2 MB/s eta 0:00:37
   ---------------------------------------- 0.5/45.3 MB 1.2 MB/s eta 0:00:37
   ---------------------------------------- 0.5/45.3 MB 1.2 MB/s eta 0:

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Alexandra\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [9]:
pip install dbr

  Obtaining dependency information for dbr from https://files.pythonhosted.org/packages/93/ce/047008311fe8cac84376f2f51d66f221bb2e688233c89a5b8e56f6b0c421/dbr-9.6.30-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.1 MB 330.3 kB/s eta 0:00:31
   ---------------------------------------- 0.0/10.1 MB 330.3 kB/s eta 0:00:31
   ---------------------------------------- 0.0/10.1 MB 330.3 kB/s eta 0:00:31
   ---------------------------------------- 0.1/10.1 MB 328.2 kB/s eta 0:00:31
   ---------------------------------------- 0.1/10.1 MB 403.5 kB/s eta 0:00:25
   ---------------------------------------- 0.1/10.1 MB 481.4 kB/s eta 0:00:21
   ---------------------------------------- 0.1/10.1 MB 481.4 kB/s eta 0:00:21
    --------------------------------------- 0.2/10.1 MB 477.7 kB/s eta 0:00:21
    -----------------------

In [4]:
# Initialize Dynamsoft Barcode Reader
reader = BarcodeReader()
# Apply for a trial license: https://www.dynamsoft.com/customer/license/trialLicense
license_key = "t0073oQAAAGLE7UzELVyejqpqOxXTTAzDS/6uAaHL2B6V2RtG1tLpAWoOwvDbneTb62NcI+mMcfvHlAyA4vKTmWyd3+t1a5ot+gYi6A=="
reader.init_license(license_key)
 
def decodeframe(frame, left, top, right, bottom):
    settings = reader.reset_runtime_settings() 
    settings = reader.get_runtime_settings()
    settings.region_bottom  = bottom
    settings.region_left    = left
    settings.region_right   = right
    settings.region_top     = top
    settings.barcode_format_ids = EnumBarcodeFormat.BF_QR_CODE
    settings.expected_barcodes_count = 1
    reader.update_runtime_settings(settings)
 
    try:
        text_results = reader.decode_buffer(frame)
        if text_results != None:
            for result in text_results:
                return result.barcode_text
    except BarcodeReaderError as bre:
        print(bre)
 
    return None

In [5]:
def obj_det(frame, threshold):
    
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    
    model.setInput(blob)
    outputs = model.forward(ln)
    
    classIds = []
    confidences = []
    boxes = []
    
    h, w = frame.shape[:2]
    
    for out in outputs:
        for detection in out:
            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > threshold:
                box = detection[:4] * np.array([w, h, w, h])
                (x, y, width, height) = box.astype("int")
                left = int(x - width / 2)
                top = int(y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, int(width), int(height)])

    indices = cv2.dnn.NMSBoxes(boxes, confidences, threshold, threshold - 0.1)
    
    if len(indices) > 0:
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            #color = [int(c) for c in colors[classIDs[i]]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            text = "{}: {:.4f}".format(classes[classIds[i]], confidences[i])
            cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
            htmlQR = decodeframe(frame, x, y, x+w, y+h)
            cv2.putText(frame, str(htmlQR), (x, y + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
      
    det = cv2.barcode.BarcodeDetector()
    (rv, detections) = det.detect(frame)
    if rv:
        for barcode in detections:
            cv2.polylines(frame, [np.int32(barcode)], isClosed=True, color=(255, 0, 0), thickness=2)

    return frame

In [8]:
classes = open('/Users/sondosali/Downloads/yolo3-tiny-qr/qrcode.names').read().strip().split('\n')
model = cv2.dnn.readNetFromDarknet('/Users/sondosali/Downloads/yolo3-tiny-qr/qrcode-yolov3-tiny.cfg', '/Users/sondosali/Downloads/yolo3-tiny-qr/qrcode-yolov3-tiny_last.weights')
model.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
ln = model.getUnconnectedOutLayersNames()

print('Classes names: ',classes)
cam = cv2.VideoCapture(0)
while (True):
    has_frame, frame = cam.read()
    
    if not has_frame:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    img_result = obj_det(frame, 0.5)
    
    cv2.imshow('frame', img_result)
    
    
    if cv2.waitKey(1) == ord('q'):
        cam.release()
        cv2.destroyAllWindows()
cam.release()
cv2.destroyAllWindows()

Classes names:  ['QR_CODE']
Can't receive frame (stream end?). Exiting ...
